# Scraping Data

This notebook will cover the web-scraping and data collection part of the project. I am getting the data from [Tim Sevenhuysen's](https://twitter.com/TimSevenhuysen) website: [Oracle's Elixir](https://oracleselixir.com/). I will scrape and store all player and match statistics from his website into sqlite3 databases. 

### League of Legends team and player statistics
Since most of the data is in HTML tables, I will first use a parser that will make it easier to get these tables. I have copied some code from [Scott Rome](http://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/) who has made an excellent and robust script to extract tables from HTML pages. Basically, we want the team and player statistics for each of the region in their own databses. This is easy to do since the links are all formatted nicely: for example, all European teams and player stats will all contain "/eu/" in the url. I have a list of such patterns for each region and stored them in the array `to_scrape`. I will create an HTML parser, give it the url and a pattern, and it will get all the tables matching that pattern and put them all in a sqlite3 database. Simple!

In [9]:
import sys
sys.path.append(".//utils//")
import sqlite3, os
import pandas as pd
from scrape_utils import HTMLTableParser
import os

In [2]:
url = "https://oracleselixir.com/statistics/player-stats/"
to_scrape = ["na","eu","lck","lms","lpl","international","cblol","tcl"]
parser = HTMLTableParser()
directory = ".\\databases"
os.chdir(directory)
for ii in to_scrape:
    db = ii+".db"
    parser.scrape_data(url,ii,db)

Creating database na.db and inserting tables...


C:\Anaconda3\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Inserted 213 tables into na.db
Creating database eu.db and inserting tables...
Inserted 99 tables into eu.db
Creating database lck.db and inserting tables...
Inserted 71 tables into lck.db
Creating database lms.db and inserting tables...
Inserted 47 tables into lms.db
Creating database lpl.db and inserting tables...
Inserted 30 tables into lpl.db
Creating database international.db and inserting tables...
Inserted 67 tables into international.db
Creating database cblol.db and inserting tables...
Inserted 30 tables into cblol.db
Creating database tcl.db and inserting tables...
Inserted 15 tables into tcl.db


Lets see what the tables are named and what one of these tables looks like:

In [3]:
conn = sqlite3.connect("na.db")
cursor = conn.cursor()
query = """SELECT * FROM 'sqlite_master' WHERE type = "table" LIMIT 5;"""
display(pd.read_sql_query(query,conn))x
query = """SELECT * FROM 'na-lcs-2018-spring-playoffs-team-statistics' LIMIT 5;"""
display(pd.read_sql_query(query,conn))

,type,name,tbl_name,rootpage,sql
0,table,na-lcs-2018-spring-playoffs-team-statistics,na-lcs-2018-spring-playoffs-team-statistics,2,"CREATE TABLE ""na-lcs-2018-spring-playoffs-team..."
1,table,na-lcs-2018-spring-regular-season-team-statistics,na-lcs-2018-spring-regular-season-team-statistics,4,"CREATE TABLE ""na-lcs-2018-spring-regular-seaso..."
2,table,na-lcs-2017-regional-qualifier-team-statistics,na-lcs-2017-regional-qualifier-team-statistics,6,"CREATE TABLE ""na-lcs-2017-regional-qualifier-t..."
3,table,na-2017-summer-playoffs-team-statistics,na-2017-summer-playoffs-team-statistics,8,"CREATE TABLE ""na-2017-summer-playoffs-team-sta..."
4,table,na-2018-spring-promotion-team-statistics,na-2018-spring-promotion-team-statistics,10,"CREATE TABLE ""na-2018-spring-promotion-team-st..."


,index,Team,GP,W,L,AGT,K,D,K:D,CKPM,...,DRG%,ELD%,FBN%,BN%,LNE%,JNG%,WPM,WCPM,VWC,IWC
0,0,100 Thieves,8.0,3.0,5.0,37.0,61.0,79.0,0.77,0.47,...,39%,50%,25%,36%,50.4%,48.9%,3.48,1.65,61.1%,21.5%
1,1,Cloud9,3.0,0.0,3.0,38.8,22.0,30.0,0.73,0.45,...,42%,50%,33%,20%,50.7%,48.1%,2.98,1.83,60.4%,25.0%
2,2,Clutch Gaming,12.0,5.0,7.0,35.0,104.0,113.0,0.92,0.52,...,40%,50%,55%,47%,48.9%,45.6%,4.29,1.39,61.8%,20.5%
3,3,Echo Fox,7.0,4.0,3.0,27.9,72.0,84.0,0.86,0.80,...,60%,-,50%,50%,50.1%,59.9%,3.57,1.05,54.5%,16.8%
4,4,Team Liquid,10.0,9.0,1.0,31.9,138.0,65.0,2.12,0.64,...,74%,50%,80%,79%,50.5%,50.0%,3.95,1.32,66.8%,23.8%


The second table shows an example of team statistics. I don't know what a lot of these columns mean so I'm also get the data dictionaries for the tables.

In [4]:
url="http://oracleselixir.com/definitions/"
conn = sqlite3.connect("data_dictionary.db")
hp = HTMLTableParser()
table = hp.parse_url(url)[0][1]
table.columns = ["Variable","Description"]
name="player_team_stats_dictionary"
display(table.head())
table.to_sql(name,con=conn,if_exists='fail')

,Variable,Description
0,GP,Games Played
1,W,Wins
2,L,Losses
3,AGT,Average Game Time (sometimes also called “G Len”)
4,P%,Percentage of games champion was picked in the...


## Match data

The team and player statistics contain average/cumulative statistics for an entire season. There is a lot more data we can get from each individual match and thankfully, it's available for download on Oracle's Elixir. So lets put the match data into SQL databases. I had to download the match data and convert it to a CSV in excel beforehand. I will just use Pandas and sqlite3 to create the SQL tables. 

In [8]:
import sqlite3, os, glob
import pandas as pd
directory = "..\\matchdata"
conn = sqlite3.connect("match_data.db")
cur = conn.cursor()
os.chdir(directory)
for ii in glob.glob("*.txt"):
    df = pd.read_csv(ii,sep="\t",encoding='latin')
    name=ii.replace(".txt","")#.split("OraclesElixir")[0].replace("a-","a")
    print(name)
    try:
        df.to_sql(name,con=conn,if_exists='fail')
    except:
        continue

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2016matchdata


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,34,35,36,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2017matchdata


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2018matchdata


There are some errors about mixed data types but we will take care of those later on! For now, lets get the data dictionary for the match data:

In [6]:
url="http://oracleselixir.com/match-data/match-data-dictionary/"
conn = sqlite3.connect("data_dictionary.db")
hp = HTMLTableParser()
table = hp.parse_url(url)[0][1]
table.columns = ["Variable","Description"]
table = table.drop(0,axis=0)
name="matches_data_dictionary"
print(table.head())
table.to_sql(name,con=conn,if_exists='fail')

  Variable                                        Description
1   gameid         Unique game identifier from Riot’s server.
2      url                                 Match history link
3   league                                             League
4    split  Time period covered, denoted by year and suffi...
5     week  Within-split week and day (“week within season...


And we're done!